BDLE 2021

date du document  :  07/10/2021 17:37

# PROJET DM1 : IMDB et YAGO (avec COLAB)


Mini-projet sur la restructuration de données et l'analyse de données.
Voir le moodle pour la date de remise et éventuelles précisions sur le travail à faire.

## Préparation

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [ ]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/ens/bdle/DM1"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

In [ ]:
drive_dir = "/content/DM1"

Installer pyspark et findspark :


In [ ]:
!pip install -q pyspark
!pip install -q findspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 52.0 MB/s 


Démarrer la session spark

In [ ]:
import os
# !find /usr/local -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1633864059187


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [ ]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
# Alternative to Databricks display function.
def display(df, n=100):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas().head(n)

In [ ]:
pd.set_option('display.max_colwidth', 80)
pd.set_option("display.max_rows", None, "display.max_columns", None)

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = val.split(";")
    for r in tabRequetes:
        if len(r) > 2:
          derniere = spark.sql(r)
    return display(derniere)

## Accès aux données

URL pour accès aux datasets

In [ ]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


#### IMDB
Télécharger les fichiers de IMDB directement sur le cluster databricks, sans passer par votre ordinateur personnel.

##### Sample: échantillon de la base IMDB

In [ ]:
import os
from urllib import request
import tarfile

imdb_dir = drive_dir 

url = PUBLIC_DATASET + "/imdb/vldb2015/csvfiles_sample001.tgz"
# temp = "/temp"
local_file = imdb_dir + "/csvfiles_sample001.tgz"

os.makedirs(imdb_dir, exist_ok=True)

if(os.path.isfile(local_file)):
  print("file is already downloaded")
else:
  print("downloading from URL: ", url, "save in : ", local_file)
  request.urlretrieve(url, local_file)
  os.chdir(imdb_dir)
  tar = tarfile.open(local_file)
  tar.extractall()
  tar.close()

# Liste des fichiers de IMDB
dir = imdb_dir + "/csvfiles_sample001/"
os.listdir(dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/imdb/vldb2015/csvfiles_sample001.tgz save in :  /content/DM1/csvfiles_sample001.tgz


['role_type.csv',
 'company_name.csv',
 'info_type.csv',
 'movie_companies.csv',
 'kind_type.csv',
 'name.csv',
 'company_type.csv',
 'movie_info.csv',
 'cast_info.csv',
 'person_info.csv',
 'title.csv']

##### Données complètes: name_all et title_all
Télécharger le fichier complet des noms de personnes et les titres de films

In [ ]:
import os
def load_IMDB_file(file):
  if(os.path.isfile( imdb_dir + "/" + file)):
    print(file, "is already stored")
  else:
    url = PUBLIC_DATASET + "/imdb/vldb2015/csvfiles/" + file
    print("downloading from URL: ", url, "save in : " + imdb_dir + "/" + file)
    request.urlretrieve(url , imdb_dir + "/"  + file)

load_IMDB_file("name.csv.gz")
load_IMDB_file("title.csv.gz")

# Liste des fichiers de IMDB
os.listdir(imdb_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/imdb/vldb2015/csvfiles/name.csv.gz save in : /content/DM1/name.csv.gz
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/imdb/vldb2015/csvfiles/title.csv.gz save in : /content/DM1/title.csv.gz


['csvfiles_sample001.tgz', 'csvfiles_sample001', 'name.csv.gz', 'title.csv.gz']

In [ ]:
# name_all

schema_name = """
  id int,
  name string,
  imdb_index string,
  imdb_id int,
  gender string,
  name_pcode_cf string,
  name_pcode_nf string,
  surname_pcode string,
  md5sum string
"""

name_all = spark.read.csv(path = imdb_dir + "/name.csv.gz", schema = schema_name)
display(name_all)

,id,name,imdb_index,imdb_id,gender,name_pcode_cf,name_pcode_nf,surname_pcode,md5sum
0,3343,"Abela, Mike",None,NaN,m,A1452,M214,A14,61f733c0298a7cb5a461fd787a655d70
1,446,"A., David",None,NaN,m,A313,D13,A,cf45e7b42fbc800c61462988ad1156d2
2,126,"-Alverio, Esteban Rodriguez",None,NaN,m,A4162,E2315,A416,f5c410bff6839b545d04c531f776e8f2
3,1678,"Abbas, Athar",None,NaN,m,A1236,A3612,A12,cf230f6ed718a330dc688c20b8d741d3
4,3610,"Aberer, Leo",None,NaN,m,A164,L16,A16,05684f16b84db387e613d3de57eca5b7
5,506,A.J.,II,NaN,m,A2,None,None,c63baf59b537e471b6ec4b68c5d1373e
6,465,"A., Narayana Rao",None,NaN,m,A5656,N656,A,c7a6b13ba56547bd0a1c352171b2d0c2
7,3612,"Aberg, Siv Marta",None,NaN,m,A1621,S1563,A162,eda04e0a59f7548878f2663eeb02164e
8,1932,"Abbot Jr., Greg",None,NaN,m,A1326,G6213,None,f3c72bdb04a43aae7af1424c261d50ff
9,2266,"Abdalla, Isam",None,NaN,m,A1342,I2513,A134,22eb3783f3ff2ec3554311899f152960


In [ ]:
# title_all

schema_title = """
          id INT, 
          title STRING, 
          imdb_index STRING, 
          kind_id INT, 
          production_year INT, 
          imdb_id INT, 
          phonetic_code STRING, 
          episode_id STRING,
          season_nr INT, 
          episode_nr INT, 
          series_years STRING, 
          md5sum STRING
        """
title_all = spark.read.csv(path = imdb_dir + "/title.csv.gz", schema = schema_title)
display(title_all)

,id,title,imdb_index,kind_id,production_year,imdb_id,phonetic_code,episode_id,season_nr,episode_nr,series_years,md5sum
0,80889,(#1.66),None,7,1980.0,NaN,None,80815,1.0,66.0,None,4de45f35edf0b753c54ab72dcbe68bb5
1,5156,Josie Duggar's 1st Shoes,None,7,2010.0,NaN,J2326,5022,4.0,13.0,None,8d492850166137b550ceb0e9a5b46086
2,197772,(#2.8),None,7,1962.0,NaN,None,197751,2.0,8.0,None,152b0f7213dd467760823b0d6387a6ec
3,111913,(2012-09-13),None,7,2012.0,NaN,None,111095,NaN,NaN,None,daf144630ef1ab1cd5d1d028b406cdcf
4,117556,(#1.1042),None,7,NaN,NaN,None,117506,1.0,1042.0,None,be62adfea75e456d47ddab5c68ac96b0
5,40704,Anniversary,None,7,1971.0,NaN,A5162,40698,4.0,9.0,None,5e12ce73fac1d1dcf94136b6e9acd8f8
6,164312,(#13.60),None,7,1997.0,NaN,None,164254,13.0,60.0,None,2d7b85508041bc8b1e4b576f328727b9
7,149337,Mellody Hobson,None,7,2005.0,NaN,M4312,149328,1.0,8.0,None,5bf869eaffa411fdc51e06c2020952d5
8,32020,Kiss Me Kate,None,7,2011.0,NaN,K2523,32014,1.0,10.0,None,293e8c75c7f35a4035abf617962be5a9
9,36858,(2012-09-11),None,7,2012.0,NaN,None,36138,NaN,NaN,None,7c4d73fbab6bf1bbf3cb3958c5f975a9


#### YAGO
Télécharger les fichiers de YAGO

In [ ]:
YAGO_dir = drive_dir + "/yago"

os.makedirs(YAGO_dir, exist_ok=True)

def yagoLoad(file):
  url = PUBLIC_DATASET + "/yago/yago4_2020/" + file
  local_file = YAGO_dir + "/" + file

  if(os.path.isfile( YAGO_dir + "/" + file)):
    print(file, "is already stored ")
  else:
    print("downloading from URL: ", url, "save in : ", local_file)
    request.urlretrieve(url, local_file)
    
yagoLoad("yago-actor-facts.snappy.orc")
yagoLoad("yago-person.snappy.orc")

# Liste des fichiers de YAGO
os.listdir(YAGO_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/yago/yago4_2020/yago-actor-facts.snappy.orc save in :  /content/DM1/yago/yago-actor-facts.snappy.orc
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/yago/yago4_2020/yago-person.snappy.orc save in :  /content/DM1/yago/yago-person.snappy.orc


['yago-person.snappy.orc', 'yago-actor-facts.snappy.orc']

##### Les personnes dans Yago

In [ ]:
yagoPerson = spark.read.orc(path = YAGO_dir + "/yago-person.snappy.orc")
display(yagoPerson)

,subject
0,<http://yago-knowledge.org/resource/Heywood_L._Edwards>
1,<http://yago-knowledge.org/resource/Ramanichandran>
2,<http://yago-knowledge.org/resource/Yegor_Kiryakov>
3,<http://yago-knowledge.org/resource/Patrick_J._Adams>
4,<http://yago-knowledge.org/resource/Basilio_J._Valdes>
5,<http://yago-knowledge.org/resource/Meryem_Uzerli>
6,<http://yago-knowledge.org/resource/Marie-Louise_Loubet>
7,<http://yago-knowledge.org/resource/Alex_Lacey>
8,<http://yago-knowledge.org/resource/Michael_Hankinson>
9,<http://yago-knowledge.org/resource/Éric_Alard>


##### Les faits des acteurs dans Yago

In [ ]:
actorFacts = spark.read.orc(path = YAGO_dir + "/yago-actor-facts.snappy.orc")
display(actorFacts)

,subject,property,object
0,<http://yago-knowledge.org/resource/40_(record_producer)>,<http://schema.org/alumniOf>,<http://yago-knowledge.org/resource/Academy_of_Art_University>
1,<http://yago-knowledge.org/resource/40_(record_producer)>,<http://schema.org/nationality>,<http://yago-knowledge.org/resource/Canada>
2,<http://yago-knowledge.org/resource/40_(record_producer)>,<http://schema.org/birthPlace>,<http://yago-knowledge.org/resource/Toronto>
3,<http://yago-knowledge.org/resource/40_(record_producer)>,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Actor>
4,<http://yago-knowledge.org/resource/40_(record_producer)>,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Songwriter>
5,<http://yago-knowledge.org/resource/40_(record_producer)>,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Child_actor>
6,<http://yago-knowledge.org/resource/A._D._Calvo>,<http://schema.org/image>,<http://commons.wikimedia.org/wiki/Special:FilePath/AD-Calvo-in-2007.jpg>
7,<http://yago-knowledge.org/resource/A._D._Calvo>,<http://schema.org/nationality>,<http://yago-knowledge.org/resource/Argentina>
8,<http://yago-knowledge.org/resource/A._D._Calvo>,<http://schema.org/nationality>,<http://yago-knowledge.org/resource/United_States>
9,<http://yago-knowledge.org/resource/A._D._Calvo>,<http://schema.org/givenName>,<http://yago-knowledge.org/resource/Alejandro>


## Définir les tables IMDB

Le dossier contenant les fichiers csv de IMDB

In [ ]:
dir = drive_dir + "/csvfiles_sample001/"

### Les tables initiales

#### Title
La table Title affecte un identifiant id à un film

In [ ]:
schema_title = """
          id INT, 
          title STRING, 
          imdb_index STRING, 
          kind_id INT, 
          production_year INT, 
          imdb_id INT, 
          phonetic_code STRING, 
          episode_id STRING,
          season_nr INT, 
          episode_nr INT, 
          series_years STRING, 
          md5sum STRING
        """

title = spark.read.csv(path = dir + "title.csv", schema = schema_title).persist()
title.createOrReplaceTempView("Title")
# display(title)




#### Kind_Type et Info_Type
* La table Kind_type
* La table Info_type indique la nature d'un attribut d'information. 

Sert dans Movie_Info, Person_Info

In [ ]:
#============
# Kind_Type
#============
schema_kind_type = "id INT, kind STRING"

kind_type = spark.read.csv(path = dir + "kind_type.csv", schema = schema_kind_type).persist()

kind_type.createOrReplaceTempView("Kind_Type")
# kind_type.printSchema()
# display(kind_type)



#==============
# Info_Type
#==============
schema_info_type = "id INT, info STRING"

info_type = spark.read.csv(path = dir + "info_type.csv", schema = schema_info_type).persist()

info_type.createOrReplaceTempView("Info_Type")
#info_type.printSchema()

display(info_type)

,id,info
0,1,runtimes
1,2,color info
2,3,genres
3,4,languages
4,5,certificates
5,6,sound mix
6,7,tech info
7,8,countries
8,9,taglines
9,10,keywords


#### Movie_info

In [ ]:
#==============
# Movie_Info
#==============
schema_movie_info = """
    id int,
    movie_id int,
    info_type_id int,
    info string,
    note string
"""

movie_info = spark.read.csv(path = dir + "movie_info.csv", schema = schema_movie_info).persist()

movie_info.createOrReplaceTempView("Movie_Info")
#movie_info.printSchema()

display(movie_info)

,id,movie_id,info_type_id,info,note
0,11658949,13,16,Italy:29 March 2012,None
1,11659887,434,16,Netherlands:7 January 2013,None
2,11661227,1680,16,USA:11 May 2011,None
3,11661228,1680,16,USA:11 May 2011,None
4,11661771,1841,16,Spain:20 March 2005,None
5,11662532,2907,16,Spain:28 September 2002,None
6,11662603,2977,16,USA:22 August 2011,None
7,11663044,3228,16,USA:27 September 1997,None
8,214865,3228,15,"Two-Tone: [watches Nanny set a yard sale box down] Yard sales! I love yard sales! Actually I hate yard sales, the pros of a yard sale are the great bargains but the cons of a yard sale are that everything is used, eww yuck::Cadpig: Two-Tone... FOCUS!",None
9,214866,3228,15,"Rolly: Anybody want a peanut?... What? It's not like I've offered you some of my food before. Okay, so it's been a while.",None


#### Cast_Info

In [ ]:
#==============
# Cast_Info
#==============
schema_cast_info = """
  id int,
  person_id int,
  movie_id int,
  person_role_id int,
  note string,
  nr_order int,
  role_id int
"""

cast_info = spark.read.csv(path = dir + "cast_info.csv", schema = schema_cast_info).persist()

cast_info.createOrReplaceTempView("Cast_Info")
#cast_info.printSchema()

display(cast_info)

,id,person_id,movie_id,person_role_id,note,nr_order,role_id
0,16255404,2202581,434,2671901.0,None,9.0,2
1,31519800,3558778,434,NaN,None,NaN,9
2,16668206,2256708,434,2264435.0,None,2.0,2
3,1148671,159670,434,271127.0,None,1.0,1
4,18308464,2464312,434,2922782.0,None,NaN,2
5,3164925,423423,434,9407.0,None,NaN,1
6,19291922,2593019,434,3040044.0,None,4.0,2
7,5264485,705755,434,1013996.0,None,3.0,1
8,19618701,2634949,434,2182050.0,None,5.0,2
9,5806077,786248,434,1107705.0,None,6.0,1


#### Role_type

In [ ]:
#=============
# Role_Type
#=============
schema_role_type = "id INT, role STRING"

role_type = spark.read.csv(path = dir + "role_type.csv", schema = schema_role_type).persist()

role_type.createOrReplaceTempView("Role_Type")
#role_type.printSchema()

display(role_type)

,id,role
0,1,actor
1,2,actress
2,3,producer
3,4,writer
4,5,cinematographer
5,6,composer
6,7,costume designer
7,8,director
8,9,editor
9,10,miscellaneous crew


#### Name
La table Name affecte un identifiant id à une personne

In [ ]:
#==============
# Name 
#==============
schema_name = """
  id int,
  name string,
  imdb_index string,
  imdb_id int,
  gender string,
  name_pcode_cf string,
  name_pcode_nf string,
  surname_pcode string,
  md5sum string
"""

name = spark.read.csv(path = dir + "name.csv", schema = schema_name).persist()

name.createOrReplaceTempView("Name")
#name.printSchema()

display(name)

,id,name,imdb_index,imdb_id,gender,name_pcode_cf,name_pcode_nf,surname_pcode,md5sum
0,333,"5, Johnny",None,NaN,m,J5,None,None,c845df6a62d46f897c7620f00f1e97a5
1,339,50 Cent,None,NaN,m,C53,None,None,faba8d4ac1b5880ef64a5de3ea7d1fa9
2,400,"A El Rahim, Ahmed",None,NaN,m,A4653,A5346,A465,9698c31ec0eb8b38172ae89e8c67b028
3,457,"A., Kodanda Rami Reddy",None,NaN,m,A2353,K3536,A,64223f8728574d3dce39126c365b5dc9
4,471,"A., Randy",None,NaN,m,A653,R53,A,b5301fd59f93e50ab8893e9b80e3a35a
5,624,"Aaker, Lee",None,NaN,m,A264,L26,A26,e15546bb152954e379a3554c28125661
6,673,"Aalto, Kari",None,NaN,m,A4326,K643,A43,8ad8a724a6ef5f40400c149162e12821
7,750,"Aames, Willie",None,NaN,m,A524,W452,A52,f983dd44bb1392c1575dfe5ffb087e1b
8,763,"Aamund, Asger",None,NaN,m,A5326,A2653,A53,0b6579030aba0724f451098c3bd533db
9,771,"Aanensen, Peter",None,NaN,m,A5251,P3652,A525,1d4412351d4d6c5cfd6eb93eea053f25


#### Person_Info

In [ ]:
#==============
# Person_Info
#==============
schema_person_info = """
  id int,
  person_id int,
  info_type_id int,
  info string,
  note string
"""

person_info = spark.read.csv(path = dir + "person_info.csv", schema = schema_person_info).persist()
person_info.createOrReplaceTempView("Person_Info")

display(person_info)

,id,person_id,info_type_id,info,note
0,302,333,33,(2012) Portlandia,None
1,303,333,26,"Martini-Connally, John",None
2,304,333,20,"Los Angeles, California, USA",None
3,305,333,17,"Has worked & studied alongside studio engineer giants like 'Kelley Baker (I)' (qv), 'Wayne Woods (I)' (qv), 'Paul Nelson (VIII)' (qv), 'John Neff' (qv), 'Russ Gorsline' (qv), 'Colin O'Neill (I)' (qv), 'Matt Meyer (II)' (qv), Brent Rogers, Randy Johnson, and 'Will Vinton' (qv).",None
4,306,333,17,Bachelors in Digital Media Production from the Art Institute of Portland.,None
5,307,333,17,Nephew of writer/director 'Richard Martini (I)' (qv).,None
6,308,333,17,Related to former Texas Governor & U.S. Secretary of the Treasury 'John Connally (I)' (qv).,None
7,309,333,21,7 September 1984,None
8,310,339,37,"""Big Fish"" (Greece), 4 June 2006, Iss. 28",None
9,311,339,37,"""Hitkrant"" (Netherlands), 19 March 2005, Iss. 11",None


#### Movie_Companies

In [ ]:
#==============
# Movie_Companies
#==============
schema_movie_companies = """
    id int,
    movie_id int,
    company_id int,
    company_type_id int,
    note string
"""

movie_companies = spark.read.csv(path = dir + "movie_companies.csv", schema = schema_movie_companies).persist()

movie_companies.createOrReplaceTempView("Movie_Companies")
#movie_companies.printSchema()

display(movie_companies)

,id,movie_id,company_id,company_type_id,note
0,1318,3342,216,1,(2000) (USA) (TV)
1,1319,3342,217,1,(200?) (Japan) (TV)
2,1454,3981,46,1,(2009) (Hungary) (TV)
3,2889,8413,423,1,(2007) (Japan) (DVD)
4,2890,8413,49,1,(1987-1990) (USA) (TV) (original airing)
5,2891,8413,428,1,(2010) (USA) (DVD)
6,1276676,8413,71104,2,(producer)
7,1276677,8413,71105,2,(in association with)
8,4284,9005,21,1,(2010) (Australia) (TV)
9,4285,9005,49,1,(2010) (USA) (TV)


#### Company_Name

In [ ]:
schema_company_name = """
    id int,
    name string,
    country_code string,
    imdb_id int,
    name_pcode_nf string,
    name_pcode_sf string,
    md5sum string
"""

company_name = spark.read.csv(path = dir + "company_name.csv", schema = schema_company_name)
company_name.createOrReplaceTempView("Company_Name")
#company_name.printSchema()

display(company_name)

,id,name,country_code,imdb_id,name_pcode_nf,name_pcode_sf,md5sum
0,1,E! Entertainment Television,[us],NaN,E5363,E5363,ddc1ca3453a33526ad931106cb60d004
1,2,YouTube,[us],NaN,Y31,Y312,3bd77ab33669d8883d345ee754928caf
2,4,5*,[gb],NaN,None,G1,2c0b57f4db6ba181ca8e7414284e9d35
3,5,Canadian Television (CTV),[ca],NaN,C5353,C5353,7bde1000f06906373c3ae6a48d981511
4,6,Columbia Broadcasting System (CBS),[us],NaN,C4516,C4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
5,7,Nine Network Australia,[au],NaN,N5362,N5362,70a4657ea90ca544e4071e363774cc7c
6,8,Veronica,[nl],NaN,V652,V6525,28f313ed51c6f708e6755e2d5d34f0a3
7,9,Warner Channel Latin America,[br],NaN,W6562,W6562,eb1f896ee609e97cfbfb007f3de1afa5
8,10,Lorimar Telepictures,[us],NaN,L6563,L6563,6c941a84e38309ddf60871f4b76d8b53
9,11,Warner Bros. Television,[us],NaN,W6561,W6561,259d3edf9bdf0b20fc2db3e494113e99


#### Company_type

In [ ]:
#==============
# Company_Type
#==============
schema_company_type = "id INT, kind STRING"

company_type = spark.read.csv(path = dir + "company_type.csv", schema = schema_company_type).persist()
company_type.createOrReplaceTempView("Company_Type")
#company_type.printSchema()

display(company_type)

,id,kind
0,1,distributors
1,2,production companies
2,3,special effects companies
3,4,miscellaneous companies


### Detail sur les personnes et les oeuvres

#### TitleDetail

In [ ]:
%%sql
create or replace temp view TitleDetail as
select t.id, t.title, t.production_year, k.kind, it.info as property, m.info as value
from Title t, Movie_Info m, Info_type it, Kind_Type k
where t.id = m.movie_id 
and m.info_type_id = it.id
and t.kind_id = k.id
;

select * 
from TitleDetail t
order by  t.production_year desc, t.title
limit 100;

,id,title,production_year,kind,property,value
0,2310404,Star Trek 3,2016,movie,countries,USA
1,2310404,Star Trek 3,2016,movie,languages,English
2,2310404,Star Trek 3,2016,movie,genres,Action
3,2310404,Star Trek 3,2016,movie,genres,Sci-Fi
4,2310404,Star Trek 3,2016,movie,genres,Adventure
5,2310404,Star Trek 3,2016,movie,color info,Color
6,2310404,Star Trek 3,2016,movie,release dates,USA:2016
7,1694099,Arclight,2015,movie,genres,Sci-Fi
8,1694099,Arclight,2015,movie,countries,USA
9,1963857,Hundred Degrees in the Shade,2015,movie,genres,Comedy


Mémoriser le résultat de TitleDetail pour éviter de le calculer plusieurs fois

In [ ]:
%%sql
cache table TitleDetail

""


#### NameDetail

In [ ]:
%%sql
create or replace temp view NameDetail as
select n.id, n.name, n.gender, i.info as property, p.info as value
from Name n, Person_Info p, Info_type i
where n.id = p.person_id
and p.info_type_id = i.id
;
 
select * 
from NameDetail n
where n.name like 'Smi%'
order by n.name, n.property
limit 100;

,id,name,gender,property,value
0,2544489,"Smid, Coosje",f,trivia,Daughter of 'Ernst Daniël Smid' (qv) and Rosemarie Giesen van der Sluis.
1,1454381,"Smid, Ernst Daniël",m,birth date,6 May 1953
2,1454381,"Smid, Ernst Daniël",m,birth notes,"Enschede, Overijssel, Netherlands"
3,1454381,"Smid, Ernst Daniël",m,magazine cover photo,"""Ncrv Gids"" (Netherlands), 17 April 1999, Iss. 16"
4,1454381,"Smid, Ernst Daniël",m,other works,Provided the singing voice of Raspoetin in the Dutch version of _Anastasia (1997)_ (qv).
5,1454381,"Smid, Ernst Daniël",m,other works,Provided the voice of Ocious P. Potter in the Dutch version of _The Borrowers (1997)_ (qv).
6,1454381,"Smid, Ernst Daniël",m,other works,Provided the voice of Thunder-Karlsson (Donderkarel) in the Dutch version of _Pippi Longstocking (1997)_ (qv).
7,1454381,"Smid, Ernst Daniël",m,other works,Provided the singing voice of Frollo in the Dutch version of _The Hunchback of Notre Dame (1996)_ (qv).
8,1454381,"Smid, Ernst Daniël",m,other works,Provided the voice of Gouverneur Ratcliffe in the Dutch version of _Pocahontas II: Journey to a New World (1998) (V)_ (qv).
9,1454381,"Smid, Ernst Daniël",m,other works,Provided the voice of Generaal in the Dutch version of _Thunderpants (2002)_ (qv).


Mémoriser le résultat de NameDetail pour éviter de le calculer plusieurs fois

In [ ]:
%%sql
cache table NameDetail;

""


## QUESTIONS

In [ ]:
#creation des tables name_all et title_all 
name_all.createOrReplaceTempView("nameAll")
title_all.createOrReplaceTempView("titleAll")

### Exercice 1: Extraction d'information

Définir ces relations apportant des informations structurées sur les personnes. Le schéma de chaque table est donné à titre indicatif et vous pouvez l'adapter et le compléter si nécessaire.

#### Question 1 : Salary
 **Salary** (id, name, titleId, titleName, salary, extra, currency) avec
  - *salary* est le salaire perçu par une personne pour un film
  - *frequency* indique si le salaire est pour le film, un épisode de la série, une semaine de tournage, etc...
  - *extra* est un complément de salaire (exple pourcentage sur le revenu du film)
  - *currency* est la devise du salaire
 

Voir la propriété *salary history*. Le champ *titleId* peut être nul si aucune correspondance n'est trouvée dans la table TitleDetail. Le cas échéant, essayer de trouver une correspondance avec le fichier *title_all* contenant tous les films.

In [ ]:

# les champs extra, frequency, currency ne sont pas remplis en prenant en compte toutes les combinaisons.
%%sql
create or replace temp view Salary as
with T1 (select n.id, n.name, regexp_extract(value, '::.*?([0-9].*?)( to| per|\\(|\\+| plus|/|$)', 1) as salary,
                     regexp_extract(n.value, '(_"|_)(.+?)("| \\()', 2) as titleName,
                     regexp_extract(value, '::.*?(/| per )(.+?)(\\(|$)', 2) as frequency ,
                     regexp_extract(value, '::.*(\\+ |plus )(.+)', 2) as extra,
                     regexp_extract(n.value, '::(.+?)[0-9]', 1) as currency 
from NameDetail n
where n.property = 'salary history'
) select t1.id, t1.name, t.id as titleId, t1.titleName, t1.salary, t1.extra, t1.frequency, t1.currency 
from T1 t1 left outer join TitleDetail t on t1.titleName = t.title;

select distinct(*)
from Salary
limit 100;

,id,name,titleId,titleName,salary,extra,frequency,currency
0,2037895,"Garner, Jennifer",NaN,13 Going on 30,"3,000,000",,,$
1,1314543,"Robbins, Peter",NaN,A Charlie Brown Christmas,125,,,$
2,1573563,"Travolta, John",NaN,A Civil Action,"20,000,000",,,$
3,112323,"Beaudine, William",NaN,A High Diver's Last Kiss,100,,week,$
4,557318,"Gibson, Hoot",NaN,A Man's Land,"16,000",,,$
5,1009669,"McConaughey, Matthew",NaN,A Time to Kill,"250,000",,,$
6,1853675,"Bullock, Sandra",NaN,A Time to Kill,"8,000,000",,,$
7,600240,"Grisham, John",NaN,A Time to Kill,"6,000,000",,,$
8,319722,"Costello, Lou",NaN,Abbott and Costello Meet Captain Kidd,"250,000",,,$
9,2097824,"Hayworth, Rita",NaN,Affair in Trinidad,"252,000",25% of the net profits,,$


In [ ]:
# la table titleall contient plusieurs id pour chaque film, pour etre coherent on prends un seul identifiant.
%%sql
select s.id, s.name, t.titleId, s.titleName, s.salary, s.extra, s.frequency, s.currency from Salary s
left outer join (select distinct title, first(id) as titleId from titleAll group by title) t
on s.titleName = t.title

,id,name,titleId,titleName,salary,extra,frequency,currency
0,2037895,"Garner, Jennifer",392300,13 Going on 30,"3,000,000",,,$
1,1314543,"Robbins, Peter",189324,A Charlie Brown Christmas,125,,,$
2,1573563,"Travolta, John",1646007,A Civil Action,"20,000,000",,,$
3,112323,"Beaudine, William",1650349,A High Diver's Last Kiss,100,,week,$
4,557318,"Gibson, Hoot",1652584,A Man's Land,"16,000",,,$
5,1009669,"McConaughey, Matthew",13273,A Time to Kill,"250,000",,,$
6,1853675,"Bullock, Sandra",13273,A Time to Kill,"8,000,000",,,$
7,600240,"Grisham, John",13273,A Time to Kill,"6,000,000",,,$
8,319722,"Costello, Lou",1662375,Abbott and Costello Meet Captain Kidd,"250,000",,,$
9,2097824,"Hayworth, Rita",1667366,Affair in Trinidad,"252,000",25% of the net profits,,$


#### Question 2 : Height
 **Height**(id, name, heightInCm) qui recense la taille des personnes de manière exploitable, en les reconvertissant en tailles en cm, alors que les entrées de la base sont  en plusieurs unités.

In [ ]:
%%sql 
create or replace temp view Heights as 
select id, name, int(if( value like "%cm%", regexp_replace(value, "cm",""), (CAST(regexp_extract(value, '([0-9])\' ([0-9])', 1) AS int) * 30.48) + 
            (CAST(regexp_extract(value, '([0-9])\' ([0-9])', 2) AS int) * 2.54 ) + if(value like "%1/2%", 1.27, 0))) as heightInCm
from NameDetail
where property = 'height'
;

select *
from Heights
order by heightInCm desc
limit 100

#### Question 3 : Magazine

 **MagazineCover**(id, name, magazine, country, day, month, year) donnant le nom du magazine dans lequel la personne est sur la couverture, avec le pays du magazine et la date de parution. Voir la propriété *magazine cover photo*.

In [ ]:
%%sql 
create or replace temp view MagazineCover as
select id, name, regexp_extract(value, '\\"(.+)\\" \\((.+)\\), (.+)', 1) as magazine, 
                  regexp_extract(value, '\\"(.+)\\" \\((.+)\\), (.+)', 2) as country, 
                  regexp_extract(value, '\\"(.+)\\" \\((.+)\\), (Iss\\..+, )?(?:([0-9]?[0-9]) )?([a-zA-Z]+)? ?([0-9]+)( |,|$)', 4) as day,
                  regexp_extract(value, '\\"(.+)\\" \\((.+)\\), (Iss\\..+, )?(?:([0-9]?[0-9]) )?([a-zA-Z]+)? ?([0-9]+)( |,|$)', 5) as month, 
                  regexp_extract(value, '\\"(.+)\\" \\((.+)\\), (Iss\\..+, )?(?:([0-9]?[0-9]) )?([a-zA-Z]+)? ?([0-9]+)( |,|$)', 6) as year
from NameDetail
where property = 'magazine cover photo'
;

select *
from MagazineCover

#### Question 4 : Trivia

Deux tables détaillant les anecdotes connues sur des personnes, voir les valeurs de la propriété *trivia* d'une personne : 
- **TitleTrivia** (fromPersonId, fromPersonName, toTitleId, toTitleName, triviaText)
- **NameTrivia** (fromPersonId,  fromPersonName, toPersonId, toPersonName, triviaText)
    
Indications :  

Pour le champ toTitleId dans TitleTrivia : si le film n'est pas référencé dans la table TitleDetail, essayer de trouver une correspondance dans le fichier contenant tous les films (title_full).

Pour le champ toPersonId dans NameTrivia : si la personne n'est pas référencée dans la table NameDetail, essayer de trouver une correspondance dans le fichier contenant toutes les personnes (name_full).

In [ ]:
#TODO
%%sql
create or replace temp view TitleTrivia as 
with T1 as (
select id as fromPersonid, name as fromPersonName, regexp_extract(value, '_(.+?) \\(', 1) as toTitleName, value as triviaText
from NameDetail t where property = "trivia" )
select * from T1 left outer join (select distinct id as toTitleId, title from TitleDetail) on toTitleName = title
;

select *
from TitleTrivia
limit 100

,fromPersonid,fromPersonName,toTitleName,triviaText,toTitleId,title
0,9496,"Adamson, Peter","""Coronation Street""","(1983) He was charged with indecently assaulting two girls at a public swimming pool. Despite being cleared by a jury, he was sacked from _""Coronation Street"" (1960)_ (qv) after he sold his story to a national newspaper. Script writers for the show killed him off when he suffered a heart attack at the wheel of his car. His exit from ""Coronation Street"" led to a collapse in his personal life. A year after the trial his wife died and by 1991 he was bankrupt. He spent last few years of his life living in Welton, Lincolnshire, far from the fame and glamour he once enjoyed.",NaN,None
1,1989,"Abbott, Bud",The Naughty Nineties,"The performance of ""Who's on First?"" in the film _The Naughty Nineties (1945)_ (qv) is considered the quintessential version of the routine, and the clip is enshrined in a looped video at the Baseball Hall of Fame in Cooperstown, New York.",NaN,None
2,1989,"Abbott, Bud",Buck Privates,"1960: He tried to form a new duo, this time with veteran comic/instrumentalist 'Candy Candido' (qv). Abbott quipped during an interview that he and Candy would star in ""Space Privates"", a futuristic version of his and 'Lou Costello (I)' (qv)'s _Buck Privates (1941)_ (qv) classic. ""Space Privates"" never materialized and the teaming was short-lived. They weren't attracting an audience, and Abbott was forced to leave after experiencing an epileptic attack while traveling to one of their personal appearances.",NaN,None
3,11201,"Adonis, Adrian",WrestleMania III,"Finished off his WWF wrestling career without his signature blonde hair after 'Ed Leslie (I)' (qv) (aka ""Brutus Beefcake"") shaved most of it off after he lost to 'Roddy Piper' (qv) during _WrestleMania III (1987) (V)_ (qv).",NaN,None
4,1521,"Abatantuono, Diego",,"Has three children: Marta (b. 1985) from his former wife Rita; Matteo (b. 1995) and Marco (b. 1997) from his companion of 20 years, 'Giulia Begnotti' (qv).",NaN,None
5,1989,"Abbott, Bud",,He was cremated and his ashes scattered in the Pacific Ocean,NaN,None
6,1989,"Abbott, Bud",,His mother was a bareback rider for the Ringling Bothers Circus.,NaN,None
7,1989,"Abbott, Bud",,"Pictured on one of five 29¢ US commemorative postage stamps celebrating famous comedians, issued in booklet form 29 August 1991. He is shown with his partner 'Lou Costello (I)' (qv). The stamp designs were drawn by caricaturist 'Al Hirschfeld' (qv). The other comedians honored in the set are 'Stan Laurel' (qv) and 'Oliver Hardy' (qv); 'Edgar Bergen' (qv) (with alter ego Charlie McCarthy); 'Jack Benny' (qv); and 'Fanny Brice' (qv).",NaN,None
8,1989,"Abbott, Bud",,"At 'Lou Costello (I)' (qv)'s insistance, the monies earned from the their act were split 60/40, favoring Abbott. Costello's reasoning was that ""comics are a dime a dozen. Good straight men are hard to find.""",NaN,None
9,1989,"Abbott, Bud",,Father of actor 'Bud Abbott Jr.' (qv),NaN,None


In [ ]:
%%sql
select * from NameDetail where property = "trivia"

,id,name,gender,property,value
0,1521,"Abatantuono, Diego",m,trivia,"He was typecast in the role of the funny southerner (""terrunciello"") in his early movies; the turning point in his career came with 'Pupi Avati' (qv)'s _Regalo di Natale (1986)_ (qv), when he showed good qualities as a dramatic actor."
1,1521,"Abatantuono, Diego",m,trivia,"Has three children: Marta (b. 1985) from his former wife Rita; Matteo (b. 1995) and Marco (b. 1997) from his companion of 20 years, 'Giulia Begnotti' (qv)."
2,1989,"Abbott, Bud",m,trivia,He was cremated and his ashes scattered in the Pacific Ocean
3,1989,"Abbott, Bud",m,trivia,His mother was a bareback rider for the Ringling Bothers Circus.
4,1989,"Abbott, Bud",m,trivia,"Pictured on one of five 29¢ US commemorative postage stamps celebrating famous comedians, issued in booklet form 29 August 1991. He is shown with his partner 'Lou Costello (I)' (qv). The stamp designs were drawn by caricaturist 'Al Hirschfeld' (qv). The other comedians honored in the set are 'Stan Laurel' (qv) and 'Oliver Hardy' (qv); 'Edgar Bergen' (qv) (with alter ego Charlie McCarthy); 'Jack Benny' (qv); and 'Fanny Brice' (qv)."
5,1989,"Abbott, Bud",m,trivia,"At 'Lou Costello (I)' (qv)'s insistance, the monies earned from the their act were split 60/40, favoring Abbott. Costello's reasoning was that ""comics are a dime a dozen. Good straight men are hard to find."""
6,1989,"Abbott, Bud",m,trivia,Father of actor 'Bud Abbott Jr.' (qv)
7,1989,"Abbott, Bud",m,trivia,"Abbott and Costello are known in Italy as ""Gianni and Pinotto"", Abbott being Gianni and Costello being Pinotto."
8,1989,"Abbott, Bud",m,trivia,"With 'Lou Costello (I)' (qv), starred on ABC (1941-1946) and NBC (1946-1949) Radio's ""The Abbott and Costello Show."""
9,1989,"Abbott, Bud",m,trivia,"A lifelong epileptic, he died of prostate cancer following two strokes."


#### Question : 5 Lien IMDB-YAGO

On suppose que certaines personnes sont présentes à la fois dans IMDB et dans YAGO mais elles ne sont pas structurées de la même façon.

5a) Compléter les Personnes existant dans NameDetail avec des informations venant de yago (voir **yagoPerson** et **actorFacts**). 
Pour cela, définir la relation qui donne le lieu de naissance et le conjoint (ou la conjointe) d'une personne :

  - **NameYago** (id, name, yagoName, yagoBirthPlace, yagoSpouse) 

5b) Compléter le contenu de NameYago pour les personnes de IMDB existant dans *name_full*.

In [ ]:
#creation des deux tables yagoPerson et actorfacts

yagoPerson.createOrReplaceTempView("yagoPerson")
actorFacts.createOrReplaceTempView("actorFacts")

In [ ]:
%%sql
-- extraction des information sans schema de yagoPerson et actorfacts
create or replace temp view yagoPersonExtracted as
select regexp_extract(subject, '.*\/(.+)>', 1) as name
from yagoPerson;

create or replace temp view actorFactsExtracted as
select regexp_extract(subject, '.*\/(.+)>', 1) as name,
      regexp_extract(property, '.*\/(.+)>', 1) as property,
      regexp_extract(object, '.*\/(.+)>', 1) as value
from actorFacts;


-- creation de la table Yago(name, birthPlace, spouse) pour faciliter la requete
create or replace temp view Yago as
select t.name, birthPlace, spouse
from yagoPersonExtracted t 
left outer join (select name, value as birthPlace from actorFactsExtracted where property = "birthPlace") t1 on t.name = t1.name 
left outer join (select name, value as spouse from actorFactsExtracted where property = "spouse") t2 on t.name = t2.name;


Requete 5a : plusieurs personne n'existe pas dans NameDetail 

In [ ]:
%%sql
create or replace temp view NameYago as 
select n.id, n.name, a.name as yagoName, a.birthPlace as yagoBirthPlace, a.spouse as yagoSpouse
from Yago a 
left outer join (select distinct id,name from NameDetail) n 
on n.name = CONCAT(regexp_extract(regexp_extract(a.name,"(.+?)(_\\(.+\\))?$",1),".*_(.+)$",1),
              ", ",regexp_extract(a.name,"^(.+?)_",1))
where a.birthPlace is not null and a.spouse is not null
;

select count(*)
from NameYago where name is not null


Requete 5b : completer avec name_full 

In [ ]:
%%sql
create or replace temp view NameYagoCompleted as 
select n.id,regexp_replace(n.name,",","") as name , a.yagoName, a.yagoBirthPlace, a.yagoSpouse 
from NameYago a left outer join (select distinct id,name from nameAll) n 
on n.name = CONCAT(regexp_extract(regexp_extract(a.yagoName,"(.+?)(_\\(.+\\))?$",1),".*_(.+)$",1),
              ", ",regexp_extract(a.yagoName,"^(.+?)_",1))
;
select count(*) from NameYagoCompleted where name is not null

### Exercice 2 : Analyse

#### Question 6 : Restructuration
L'objectif de cette question est de savoir définir une structure complexe contenant des attributs qui ne sont pas des valeurs simples mais les listes, des tuples, des listes de tuples, ...

Définir une table **Film** (id, titre, roles, paysSalles) avec 
 - *roles* est une liste de tuples (acteur, nom, rang) où rang est le numéro d'ordre du rôle (attribu *nr_order*)
 - *paysSalles* est la liste des pays où le film est sorti en salles, triée par revenu dans le pays

In [ ]:
%%sql 

#Todo sorting by revenue 

create or replace temporary view Film as
with T1 as (
select movie_id as id , collect_list((person_id,(select first(name) from Name where id = person_id),nr_order)) as roles 
from Cast_Info
GROUP BY movie_id),
T2 as (
select id as id , collect_list(distinct(regexp_extract(value,"(.+):",1))) as paysSalles 
from TitleDetail where property = "release dates"
GROUP BY id)
select  t.id , r.title, t.roles, t2.paysSalles from T1 t, T2 t2, Title r where t.id = t2.id and r.id = t.id;

select * from Film

#### Question 7 : Fenêtres
On considère la table **RevenuWeekEndFilm**(id, titre, pays, jour, mois, année, revenu, écrans) dont les informations proviennent de la propriété *weekend gross* de TitleDetail
* un week-end est défini par les attributs (jour, mois, année)
* l'attribut *écrans* correspond à la quantité associée au terme *screens*

Exprimer les requêtes :
 - 7a) Pour chaque mois, le revenu total des films sortis aux USA durant les 3 derniers mois. Le schéma du résultat est : 
     - (mois, année, revenuMoisCourant, revenu3DerniersMois)
 - 7b) Pour chaque pays, le classement des films par revenu total. Le schéma du résultat est :
      - (pays, id, titre, revenuTotal, classement)
 - 7c) Pour chaque film, la date de début et de fin des périodes où les revenus du week end représentent au moins 10% du revenu total annuel du film. Le schéma du résultat est :
     - (id, titre, revenuAnnuel, annee, mois, jour, revenuWeekEnd, numéroPériode)

In [ ]:
%%sql

create or replace temporary view RevenuWeekEndFilm as
select id,title, regexp_extract(value,"(.+?) \\((.+?)\\) \\((.+?)\\)( \\((.+)screens\\))?",2) as pays,
                 regexp_extract(value,"(.+?) \\((.+?)\\) \\((.+?) (.+?) (.+?)\\)( \\((.+)screens\\))?",3) as jour,
                 regexp_extract(value,"(.+?) \\((.+?)\\) \\((.+?) (.+?) (.+?)\\)( \\((.+)screens\\))?",4) as mois,
                 regexp_extract(value,"(.+?) \\((.+?)\\) \\((.+?) (.+?) (.+?)\\)( \\((.+)screens\\))?",5) as annee,
                 regexp_extract(value,"(.+?) \\((.+?)\\) \\((.+?)\\)( \\((.+)screens\\))?",1) as revenu,
                regexp_extract(value,"(.+?) \\((.+?)\\) \\((.+?)\\)( \\((.+)screens\\))?",5) as ecrans
 from TitleDetail where property = "weekend gross" order by id;

 select * from RevenuWeekEndFilm

In [ ]:
# 7a

%%sql
with T1 as (
select mois,int(annee),regexp_replace(revenu,"[^0-9]","") as revenuMoisCourant, to_date(CONCAT(mois,annee),"MMMMyyyy") as date 
from RevenuWeekEndFilm
where pays = "USA")
select mois,annee, revenuMoisCourant,
  SUM(revenuMoisCourant) OVER(
    ORDER BY months_between(date, '1900-01-01')
    RANGE BETWEEN 2 PRECEDING AND current row
  ) as revenu3DerniersMois from T1 order by date

In [ ]:
#7b

%%sql
with T1 as (
select pays,id, title, sum(regexp_replace(revenu,"[^0-9]","")) as revenuTotal from RevenuWeekEndFilm group by pays,id,title)
select *, rank() over (partition by pays order by revenuTotal) as classement from T1 t order by pays, classement

In [ ]:
#7c
# chaque pays a une devise differente on ignore cette information pour faciliter les calcules

%%sql 
with T1 as (
select id, title, sum(regexp_replace(revenu,"[^0-9]","")) over (partition by id,title,annee) as revenuAnnuel, annee,mois,jour,
sum(regexp_replace(revenu,"[^0-9]","")) over (partition by id,title,annee,mois,jour) as revenuWeekEnd from RevenuWeekEndFilm),

T2 as (
select distinct(*) , sum(case when regexp_replace(revenuWeekEnd,"[^0-9]","") >= revenuAnnuel * 0.1 then 0 else 1 end) over (partition by id,title order by to_date(CONCAT(jour,mois,annee),"dMMMMyyyy")) as ok2
from T1),

T3 as (
select *, dense_rank() over (partition by id,title order by ok2) as ok3,

min(to_date(CONCAT(jour,mois,annee),"dMMMMyyyy")) OVER (PARTITION BY id,title,ok2 ) AS first , 
max(to_date(CONCAT(jour,mois,annee),"dMMMMyyyy")) OVER (PARTITION BY id,title,ok2 ) AS last
 from T2 where regexp_replace(revenuWeekEnd,"[^0-9]","") >= revenuAnnuel * 0.1
)

select id,title,revenuAnnuel,annee,mois,jour,revenuWeekEnd, ok3 as numeroPeriode 
from T3 where to_date(CONCAT(jour,mois,annee),"dMMMMyyyy") = first or to_date(CONCAT(jour,mois,annee),"dMMMMyyyy") = last


,id,title,revenuAnnuel,annee,mois,jour,revenuWeekEnd,numeroPeriode
0,1635380,102 Dalmatians,43338350.0,2000,November,26,19883351.0,1
1,1635380,102 Dalmatians,12848606.0,2001,March,25,150441.0,1
2,1640952,39 Pounds of Love,18964.0,2005,November,27,4217.0,1
3,1640952,39 Pounds of Love,18964.0,2005,December,11,6532.0,1
4,1640952,39 Pounds of Love,1317.0,2006,January,15,1227.0,2
5,1640952,39 Pounds of Love,1317.0,2006,January,22,90.0,2
6,1654259,A Nightmare on Elm Street: The Dream Child,20398441.0,1989,August,13,8115176.0,1
7,1654259,A Nightmare on Elm Street: The Dream Child,20398441.0,1989,September,17,327288.0,1
8,1654259,A Nightmare on Elm Street: The Dream Child,20398441.0,1989,October,1,241955.0,2
9,1654259,A Nightmare on Elm Street: The Dream Child,1851493.0,1990,February,14,64575.0,3
